In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from teleconnections import Teleconnection
from aqua import slurm

In [4]:
slurm.job(loglevel='DEBUG')

2023-10-13 12:15:12 :: create_folder :: WARNING  -> Folder ./slurm already exists
2023-10-13 12:15:12 :: create_folder :: WARNING  -> Folder ./slurm/logs already exists
2023-10-13 12:15:12 :: create_folder :: WARNING  -> Folder ./slurm/output already exists


2023-10-13 12:15:13 :: slurm :: INFO     -> Submitting the job to the SLURM queue
2023-10-13 12:15:13 :: slurm :: WARNING  -> #!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p small
#SBATCH -A project_465000454
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=10G
#SBATCH -t 02:30:00
#SBATCH --error=./slurm/logs/dask-worker-%j.err
#SBATCH --output=./slurm/output/dask-worker-%j.out

/users/nurissom/mambaforge/aqua_common/bin/python -m distributed.cli.dask_worker tcp://193.167.209.163:39159 --nthreads 1 --memory-limit 9.31GiB --name dummy-name --nanny --death-timeout 60

2023-10-13 12:15:13 :: slurm :: WARNING  -> #!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p small
#SBATCH -A project_465000454
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=10G
#SBATCH -t 02:30:00
#SBATCH --error=./slurm/logs/dask-worker-%j.err
#SBATCH --output=./slurm/output/dask-worker-%j.out

/users/nurissom/mambaforge/aqua_common/bin/python -m distributed.cli.dask_worker tcp://193.167.209.163:39

In [5]:
model = 'ERA5'
exp = 'era5'
source = 'monthly'

In [6]:
tc = Teleconnection(model, exp, source, 'ENSO')

2023-10-13 12:15:26 :: Teleconnection :: WARNING  -> No regridding will be performed
2023-10-13 12:15:26 :: Teleconnection :: WARNING  -> No time aggregation will be performed
2023-10-13 12:15:26 :: Teleconnection :: WARNING  -> No time aggregation will be performed


In [7]:
tc.run()

2023-10-13 12:15:34 :: eccodes :: WARNING  -> ShortName sst have multiple grib codes associated: ['34', '151159']
2023-10-13 12:15:34 :: eccodes :: WARNING  -> AQUA will take the first so that sst -> 34, please set up a correct fix if this does not look right
2023-10-13 12:15:34 :: eccodes :: WARNING  -> AQUA will take the first so that sst -> 34, please set up a correct fix if this does not look right


In [5]:
import os

import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
import matplotlib.pyplot as plt
import numpy as np

from teleconnections.plots import plot_box, add_cyclic_lon, evaluate_colorbar_limits

In [6]:
tc.regression

<xarray.DataArray 'sst' (lat: 721, lon: 1440)>
dask.array<truediv, shape=(721, 1440), dtype=float64, chunksize=(721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
  * lon      (lon) float64 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
Attributes:
    long_name:  Sea surface temperature
    units:      K
    code:       34
    table:      128
    history:    2023-10-13 11:50:02 variable renamed by AQUA fixer;\n
    paramId:    34
    cfVarName:  sst
    shortName:  sst

In [7]:
tc.regression.plot()

MemoryError: Unable to allocate 95.1 MiB for an array with shape (12, 721, 1440) and data type float64

In [13]:
def simple_map(infile):

    # Set up the projection that will be used for plotting
    proj = ccrs.PlateCarree()

    # Set up the figure and axis
    fig = plt.figure(figsize=(8, 4))

    ax = fig.add_subplot(1, 1, 1, projection=proj)

    # Add cyclic point
    data = add_cyclic_lon(infile)

    # Plot the data
    cs = ax.contourf(data['lon'], data['lat'], data['data'], cmap='RdBu_r', transform=proj)

In [14]:
simple_map(tc.regression)